# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment:  Regression Neural Network**

**Student Name: Your Name**

# Assignment Instructions

For this assignment you will use the **reg-30-spring-2018.csv** dataset.  This is a dataset that I generated specifically for this semester.  You can find the CSV file in the **data** directory of the class GitHub repository here: [reg-30-spring-2018.csv](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/reg-30-spring-2018.csv).

For this assignment you will train a neural network and return the predictions.  You will submit these predictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Normalize all numeric to zscores and all text/categorical to dummies.  Do not normalize the *target*.
* If you find any missing values (NA's), replace them with the median values for that column.
* No need for any cross validation or holdout.  Just train on the entire data set for 250 steps.
* You might get a warning, such as **"Warning: The mean of column pred differs from the solution file by 2.39"**.  Do not worry about small values, it would be very hard to get exactly the same result as I did.
* Your submitted dataframe will have these columns: id, pred.


# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [23]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [55]:
import os
import io
import requests
import pandas as pd
import numpy as np

from scipy.stats import zscore

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.model_selection import KFold

# This is your student key that I emailed to you at the beginnning of the semester.
key = "UjhQvgInJx71GabltZtqy6O1LdzxtjcE5idLxF3K"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class1_intro_python.ipynb'  # Windows
# file='/Users/jeff/projects/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # Mac/Linux
file = '/Users/jwalker/git/t81_558_deep_learning/assignments/assignment_jwalker_class4.ipynb'

# Begin assignment
path = "../data/"

filename_read = os.path.join(path,"reg-30-spring-2018.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# all numeric columns (except id)
for column in ['distance','height','landings','number','pack','age','usage','weight','volume','width','max','power','size',]:
    missing_median(df,column)
    # do NOT normalize target
    if (column != 'target'):
        encode_numeric_zscore(df,column)

# all text/categorical columns
# Should I use encode_text_index for 'item'?
for column in ['region', 'item']:
    encode_text_dummy(df,column)

# Encode the feature vector
ids = df['id']
df.drop('id',1,inplace=True)
x,y = to_xy(df,'target')

model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=250)

pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

pred = pd.DataFrame(pred)
pred.rename(columns={0: 'pred'},inplace=True)
submit_df = pd.concat([ids,pred],axis=1)
#submit_df[0:10]

submit_df.to_csv('4.csv',index=False) 

# Submit the assignment
submit(source_file=file,data=submit_df,key=key,no=4)

Epoch 1/250
 - 2s - loss: 12808.0729
Epoch 2/250
 - 0s - loss: 12798.3073
Epoch 3/250
 - 0s - loss: 12789.8576
Epoch 4/250
 - 0s - loss: 12781.6783
Epoch 5/250
 - 0s - loss: 12770.2674
Epoch 6/250
 - 0s - loss: 12755.5418
Epoch 7/250
 - 0s - loss: 12735.5570
Epoch 8/250
 - 0s - loss: 12711.0122
Epoch 9/250
 - 0s - loss: 12680.0604
Epoch 10/250
 - 0s - loss: 12642.1571
Epoch 11/250
 - 0s - loss: 12599.3761
Epoch 12/250
 - 0s - loss: 12549.4917
Epoch 13/250
 - 0s - loss: 12497.0815
Epoch 14/250
 - 0s - loss: 12438.9693
Epoch 15/250
 - 0s - loss: 12378.5541
Epoch 16/250
 - 0s - loss: 12314.7916
Epoch 17/250
 - 0s - loss: 12255.9412
Epoch 18/250
 - 0s - loss: 12203.3773
Epoch 19/250
 - 0s - loss: 12159.7963
Epoch 20/250
 - 0s - loss: 12115.4553
Epoch 21/250
 - 0s - loss: 12079.3496
Epoch 22/250
 - 0s - loss: 12055.4859
Epoch 23/250
 - 0s - loss: 12026.2337
Epoch 24/250
 - 0s - loss: 11996.3758
Epoch 25/250
 - 0s - loss: 11971.2102
Epoch 26/250
 - 0s - loss: 11948.4447
Epoch 27/250
 - 0s - 

 - 0s - loss: 4583.4537
Epoch 218/250
 - 0s - loss: 4570.8740
Epoch 219/250
 - 0s - loss: 4526.3348
Epoch 220/250
 - 0s - loss: 4516.4377
Epoch 221/250
 - 0s - loss: 4464.7110
Epoch 222/250
 - 0s - loss: 4463.1406
Epoch 223/250
 - 0s - loss: 4424.6571
Epoch 224/250
 - 0s - loss: 4378.6762
Epoch 225/250
 - 0s - loss: 4366.3121
Epoch 226/250
 - 0s - loss: 4330.4954
Epoch 227/250
 - 0s - loss: 4302.3244
Epoch 228/250
 - 0s - loss: 4240.2681
Epoch 229/250
 - 0s - loss: 4246.7822
Epoch 230/250
 - 0s - loss: 4240.0221
Epoch 231/250
 - 0s - loss: 4184.9194
Epoch 232/250
 - 0s - loss: 4158.1685
Epoch 233/250
 - 0s - loss: 4109.9887
Epoch 234/250
 - 0s - loss: 4079.9944
Epoch 235/250
 - 0s - loss: 4086.9307
Epoch 236/250
 - 0s - loss: 4017.5641
Epoch 237/250
 - 0s - loss: 4004.4452
Epoch 238/250
 - 0s - loss: 3978.1310
Epoch 239/250
 - 0s - loss: 3946.8418
Epoch 240/250
 - 0s - loss: 3913.6142
Epoch 241/250
 - 0s - loss: 3903.0916
Epoch 242/250
 - 0s - loss: 3868.1084
Epoch 243/250
 - 0s - loss